In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import selenium_base
import json
from tqdm import tqdm


In [1]:

driver = selenium_base.chrome()


data = []
for i in tqdm(range(1, 518)):
    driver.get(f'https://www.curseforge.com/minecraft/mc-mods?filter-game-version=2020709689:8203&filter-sort=4&page={i}')
    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".project-listing-row"))
    )
    for element in elements:
        image = ''
        try:
            image = element.find_element(By.CSS_SELECTOR, ".mx-auto").get_attribute('src')
        except:
            pass
        data.append({
            'image' : image,
            'link': element.find_element(By.CSS_SELECTOR, ".my-auto").get_attribute('href'),
            'name': element.find_element(By.CSS_SELECTOR, ".my-auto h3").get_attribute('innerText'),
            'description': element.find_element(By.CSS_SELECTOR, ".text-sm.leading-snug").get_attribute('innerText')
        })

    with open('mods.json', 'w+') as file:
        file.write(json.dumps(data))

100%|██████████| 517/517 [1:03:27<00:00,  7.36s/it]


In [3]:
import file_download
import selenium_base
def download(url):
    driver = selenium_base.chrome()
    driver.get(url + '/relations/dependencies')
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.project-avatar.project-avatar-64'))
        )
        elements = driver.find_elements(By.CSS_SELECTOR, '.flex.flex-col .items-end.hidden a:not(.text-base.leading-normal.font-bold)')
        for element in elements:
            download(element.get_attribute('href'))
    except Exception as e:
        pass
    page = 1
    while True:
        driver.get(url + f'/files/all?page={page}')
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".listing-body table tbody tr"))
        )
        for element in elements:
            if(element.find_element(By.CSS_SELECTOR, '.mr-2').get_attribute('innerText') == '1.16.5'):
                name = element.find_element(By.CSS_SELECTOR, '[data-action="file-link"]').get_attribute('innerText')
                link = element.find_element(By.CSS_SELECTOR, '[data-tooltip="Download file"]').get_attribute('href')+'/file'
                file_download.download(
                    link,
                    r"C:\xampp\htdocs\minecraft\mods"
                )
                driver.close()
                return name
        page += 1

print(download('https://www.curseforge.com/minecraft/mc-mods/industrial-foregoing'))

industrial-foregoing-1.16.5-3.2.14.7-16.jar
